With this analyzation, the approach is to get the sales by province for each year. We do this through a sql query. We create a dataframe , and pass in to the tables function our table column indexes, and the province and using pandas and LinearRegression from sklearn, we get the future predicted sales for 2023 and 2024. They are then outputted and printed as tables.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pyodbc

try:
    cnx = pyodbc.connect("""DRIVER={ODBC Driver 17 for SQL Server};
                            SERVER=cosc304.ok.ubc.ca;
                            DATABASE=analysis;
                            UID=cosc304;
                            PWD=cosc304""")
    cursor = cnx.cursor()
    cursor.execute("""  SELECT 
                            c.state,
                            SUM(op.quantity)*SUM(op.price) as sales,
                            YEAR(os.orderDate) as year
                        FROM customer c JOIN incart i ON c.customerId=i.customerId
                                        JOIN orderproduct op ON i.productId=op.productId
                                        JOIN ordersummary os ON op.orderId=os.orderId
                        GROUP BY c.state, YEAR(os.orderDate)
                        HAVING YEAR(os.orderDate) BETWEEN 2020 AND 2023
                        ORDER BY c.state""")

    data = {'sales': [],
            'province': [],
            'year': []}
    df = pd.DataFrame(data)

    i = 0
    for row in cursor:
        df.loc[i] = [str(row[0]), str(row[1]), str(row[2])]
        i=i+1
    
    def tables(df, province, row1, row2):
        table = df[row1:row2]
        model = LinearRegression()
        model.fit(table[['province','year']], table['sales'])
        test_2023 = pd.DataFrame({'province':[province], 'year':[2023]})
        test_2024 = pd.DataFrame({'province':[province], 'year':[2024]})
        sales_2023 = model.predict(test_2023)
        sales_2024 = model.predict(test_2024)
        row1 = pd.DataFrame({'sales':sales_2023, 'province':province, 'year':2023})
        row2 = pd.DataFrame({'sales':sales_2024, 'province':province, 'year':2024})
        table = pd.concat([table, row1], ignore_index=True)
        table = pd.concat([table, row2], ignore_index=True)
        return table
    
    print(tables(df, 'AB', 0, 3))
    print(tables(df, 'BC', 4, 7))
    print(tables(df, 'MB', 8, 11))
    print(tables(df, 'NF', 12, 15))
    print(tables(df, 'NS', 16, 19))
    print(tables(df, 'PE', 20, 23))
    print(tables(df, 'QC', 24, 27))

except pyodbc.Error as err:
    print(err)
finally:
    cnx.close()

Based on the data provided...